### 1. Prepare Data

In [ ]:
import os
import math
import copy
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from nltk import word_tokenize
from collections import Counter
from torch.autograd import Variable

In [ ]:
# initialize the parameters

UNK = 0  # id for unknown word
PAD = 1  # id for padding
BATCH_SIZE = 64 
EPOCHS = 20 
LAYERS = 6
H_NUM = 8  # multihead attention hidden size
D_MODEL = 256  # embedding size
D_FF = 1024  # feed forward fc layer dimension
DROPOUT = 0.1  
MAX_LENGTH = 60  # max length of the sentence

In [ ]:
TRAIN_FILE = '/train.txt'  
DEV_FILE = "/dev.txt"  
SAVE_FILE = '/model.pt'


In [ ]:
# TRAIN_FILE = 'nmt/en-cn/train.txt'  
# DEV_FILE = "nmt/en-cn/dev.txt"  
# SAVE_FILE = 'save/model.pt'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 2. Data Preprocessing

### Text preprocessing
####   Use padding to make sure all sentences are the same length

In [ ]:
def seq_padding(X, padding=0):

    # get the length of each sentence
    L = [len(x) for x in X]
    # get the max len
    ML = max(L)
    
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])

#### Convert the traditional chinese into simplified chinese

In [ ]:
def cht_to_chs(sent):
    sent = Converter("zh-hans").convert(sent)
    sent.encode("utf-8")
    return sent

#### Prepare the data by:
1. loading the data
2. building the vocab
3. converting the word to id
4. splitting the batch

In [ ]:
class PrepareData:
    
    def __init__(self, train_file, dev_file):
        # load the data
        self.train_en, self.train_cn = self.load_data(train_file)
        self.dev_en, self.dev_cn = self.load_data(dev_file)

        # build the vocab
        self.en_word_dict, self.en_total_words, self.en_index_dict = self.build_dict(self.train_en)
        self.cn_word_dict, self.cn_total_words, self.cn_index_dict = self.build_dict(self.train_cn)

        # word to id
        self.train_en, self.train_cn = self.wordToID(self.train_en, self.train_cn, self.en_word_dict, self.cn_word_dict)
        self.dev_en, self.dev_cn = self.wordToID(self.dev_en, self.dev_cn, self.en_word_dict, self.cn_word_dict)

        # split the batch
        self.train_data = self.splitBatch(self.train_en, self.train_cn, BATCH_SIZE)
        self.dev_data = self.splitBatch(self.dev_en, self.dev_cn, BATCH_SIZE)

    def load_data(self, path):
        """
        load the data and add BOS/EOS tag at the start/end of each sentence
        """
        en = []
        cn = []

        with open(path, 'r', encoding='utf-8') as fin:
            for line in fin:
                list_arr = line.split('\t')
                en_sentence = list_arr[0]
                en.append(['BOS'] + word_tokenize(en_sentence.lower()) + ['EOS'])
                cn.append(['BOS'] + word_tokenize(" ".join(list_arr[1])) + ['EOS'])
        
        return en, cn
    
    def build_dict(self, sentences, max_words = 50000):

        word_count = Counter()

        for sentence in sentences:
            for s in sentence:
                word_count[s] += 1
        # keep the top n most common words, n = max_words
        ls = word_count.most_common(max_words)
        
        total_words = len(ls) + 2

        word_dict = {w[0]: index + 2 for index, w in enumerate(ls)}
        word_dict['UNK'] = UNK
        word_dict['PAD'] = PAD
        
        index_dict = {v: k for k, v in word_dict.items()}

        return word_dict, total_words, index_dict

    def wordToID(self, en, cn, en_dict, cn_dict, sort=True):

        # the length of the ENG text
        length = len(en)
        
        # word to id
        out_en_ids = [[en_dict.get(word, 0) for word in sentence]for sentence in en]
        out_cn_ids = [[cn_dict.get(word, 0) for word in sentence]for sentence in cn]

        # sort the sentences by length
        def len_argsort(seq):

            return sorted(range(len(seq)), key=lambda x: len(seq[x]))

        # sort CN and ENG sentences
        if sort:
            sorted_index = len_argsort(out_en_ids)
            
            out_en_ids = [out_en_ids[s_idx] for s_idx in sorted_index]
            out_cn_ids = [out_cn_ids[s_idx] for s_idx in sorted_index]
            
        return out_en_ids, out_cn_ids

    def splitBatch(self, en, cn, batch_size, shuffle=True):

        # [0, 1, ..., len(en)-1]
        # get each batch by chunking the data into same batch_size
        idx_list = np.arange(0, len(en), batch_size)
        # shuffle
        if shuffle:
            np.random.shuffle(idx_list)
        batch_indexs = []
        for idx in idx_list:
            """
                [array([4, 5, 6, 7]), 
                 array([0, 1, 2, 3]), 
                 array([8, 9, 10, 11]),
                 ...]
            """
            batch_indexs.append(np.arange(idx, min(idx + batch_size, len(en))))

        batches = []
        for batch_index in batch_indexs:
            # get the word id
            batch_en = [en[index] for index in batch_index]  
            batch_cn = [cn[index] for index in batch_index]
            # padding into same length
            # batch × batch_size × max_len
            batch_cn = seq_padding(batch_cn)
            batch_en = seq_padding(batch_en)
            batches.append(Batch(batch_en, batch_cn))

        return batches

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        # Embedding layer
        self.lut = nn.Embedding(vocab, d_model)
        # Embedding dimension
        self.d_model = d_model

    def forward(self, x):
        # return and multiply by math.sqrt(d_model)
        return self.lut(x) * math.sqrt(self.d_model)

### Positional Encoding

Accoding to the paper **attention is all you need**（ https://arxiv.org/pdf/1706.03762.pdf , the model uses $sine$ and $cosine$ functions as an indicator of the position in sentence：   
  
$$PE_{(pos,2i)} = sin(pos / 10000^{2i/d_{\text{model}}}) \quad \quad PE_{(pos,2i+1)} = cos(pos / 10000^{2i/d_{\text{model}}})\tag{eq.1}$$  

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # initialize a tensor for positional embedding
        pe = torch.zeros(max_len, d_model, device=DEVICE)

        position = torch.arange(0., max_len, device=DEVICE).unsqueeze(1)
        # calculate the div
        div_term = torch.exp(torch.arange(0., d_model, 2, device=DEVICE) * -(math.log(10000.0) / d_model))
        
        # calculate the positional embedding
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # add one layer，(1, max_len, embedding_size)
        pe = pe.unsqueeze(0) 
        self.register_buffer('pe', pe)

    def forward(self, x):
        # sum the batch sentence embedding with positional embedding
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

### self attention

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    #  the dimension of the key vectors
    d_k = query.size(-1)
    
    # transpose the dimentions of the key vector 
    # and cal the dot product of query and key and then divide by square root of d_k
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    
    # replace 0 with large negative number
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
        
    p_attn = F.softmax(scores, dim = -1)
    
    if dropout is not None:
        p_attn = dropout(p_attn)
    
    # return the dot product of the score and value, and attention matrix
    return torch.matmul(p_attn, value), p_attn


class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()

        assert d_model % h == 0
        # head dimension
        self.d_k = d_model // h
        # number of heads
        self.h = h
        # initiate 4 fc layers
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        
        nbatches = query.size(0)

        # multiply the embeddings with query, vector and key
        # and devide them into h heads
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) 
                             for l, x in zip(self.linears, (query, key, value))]

        # return the multi attention heads and attention matrix
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        # concat the attention heads
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)

        return self.linears[-1](x)

### Attention Mask

In [ ]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        
        src = torch.from_numpy(src).to(DEVICE).long()
        trg = torch.from_numpy(trg).to(DEVICE).long()
        self.src = src
        # 
        # 1×seq length
        self.src_mask = (src != pad).unsqueeze(-2)
        # if t rg is not None，then mask the target text in decoder
        if trg is not None:
            # decoder target
            self.trg = trg[:, :-1]
            # decoder target output
            self.trg_y = trg[:, 1:]
            # target attention mask
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    # Mask
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

### Layer Normalization

$$LayerNorm(x)=\alpha \odot \frac{x_{ij}-\mu_{i}}
{\sqrt{\sigma^{2}_{i}+\epsilon}} + \beta \tag{eq.5}$$ 

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        # initiate α and β
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        # smoothing parameter
        self.eps = eps
    
    def forward(self, x):
        mean = x.mean(-1, keepdim = True)
        std = x.std(-1, keepdim = True)
        div = torch.sqrt(std*std+self.eps)
        
        out = self.a_2*((x - mean)/div) + self.b_2
        return out

#### Actually there is a function in Pytorch called ***nn.LayerNorm*** which we can simply use for layer normalization

### SubLayer

Each sub-layer (self-attention, ffnn) in each encoder has a residual connection around it, and is followed by a layer-normalization step. Which means we will add the result and the value before this result together.
$$X + SubLayer(X) $$  

In [ ]:
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        _x = self.norm(x)
        _x = sublayer(_x)
        _x = self.dropout(_x)
    
        return x + _x

### Transformer Encoder

Here is the summary of the tranformer encoder block:

1. **Word embedding and positional embedding**
$$X = EmbeddingLookup(X) + PositionalEncoding \tag{eq.2}$$
$$X \in \mathbb{R}^{batch \ size  \ * \  seq. \ len. \  * \  embed. \ dim.} $$ 

2. **Self Attention**
$$Q = Linear(X) = XW_{Q}$$ 
$$K = Linear(X) = XW_{K} \tag{eq.3}$$
$$V = Linear(X) = XW_{V}$$
$$X_{attention} = SelfAttention(Q, \ K, \ V) \tag{eq.4}$$ 

3. **Layer Normalization and sub layer**
$$X_{attention} = LayerNorm(X_{attention}) \tag{eq. 5}$$
$$X_{attention} = X + X_{attention} \tag{eq. 6}$$ 

4. **FeedForward and Relu**
$$X_{hidden} = Linear(Activate(Linear(X_{attention}))) \tag{eq. 7}$$ 

5. **Repeat step3**
$$X_{hidden} = LayerNorm(X_{hidden})$$
$$X_{hidden} = X_{attention} + X_{hidden}$$
$$X_{hidden} \in \mathbb{R}^{batch \ size  \ * \  seq. \ len. \  * \  embed. \ dim.} $$

In [ ]:
def clones(module, N):
    
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

### FeedForward

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = F.relu(self.w_1(x))
        out = self.dropout(out)
        out = self.w_2(out)
        
        return out  

### Encoder layer N=6

In [ ]:
class Encoder(nn.Module):
    # layer = EncoderLayer
    # N = 6
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        # clone N encoder layers
        self.layers = clones(layer, N)
        # Layer Norm
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        
        return self.norm(x)

Every Encoder Block has two sub layers:
    1. Attention layer
    2. Position-wise Feed Forwrd NN

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout),2)
        self.size = size
    
    def forward(self, x, mask):
        # self attention on embedding layer
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        
        return self.sublayer[1](x, self.feed_forward)

### Decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        # self-attention
        self.self_attn = self_attn
        # context attention from Encoder
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        # Context-Attention：context-attention q： decoder hidden，k,v： encoder hidden
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)
        

In [ ]:
def subsequent_mask(size):
    "Mask out subsequent positions."

    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

### Transformer Model

In [ ]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        memory = self.encode(src, src_mask)
        return self.decode(memory, src_mask, tgt, tgt_mask)

In [ ]:
class Generator(nn.Module):
    # vocab: tgt_vocab
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        # fc layer and reshape to vocab dim size
        self.proj = nn.Linear(d_model, vocab)
    
    def forward(self, x):
        # softmax
        out = F.log_softmax(self.proj(x), dim=-1)
        return out

### Define the Hyper Parameter and the Model

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h = 8, dropout=0.1):
    c = copy.deepcopy
    
    # attention
    attn = MultiHeadedAttention(h, d_model).to(DEVICE)
    # feed forward
    ff = PositionwiseFeedForward(d_model, d_ff, dropout).to(DEVICE)
    # positional encoding
    position = PositionalEncoding(d_model, dropout).to(DEVICE)
    # Transformer model
    model = Transformer(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout).to(DEVICE), N).to(DEVICE),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout).to(DEVICE), N).to(DEVICE),
        nn.Sequential(Embeddings(d_model, src_vocab).to(DEVICE), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab).to(DEVICE), c(position)),
        Generator(d_model, tgt_vocab)).to(DEVICE)
    
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            # 这里初始化采用的是nn.init.xavier_uniform
            nn.init.xavier_uniform_(p)
    return model.to(DEVICE)

### Model Training

#### Label Smoothing

In [ ]:
class LabelSmoothing(nn.Module):
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction='sum')
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size # vocab size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

In [ ]:
class SimpleLossCompute:
    
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.data.item() * norm.float()

### Optimizer

In [ ]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * (self.model_size ** (-0.5) * min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [ ]:
def run_epoch(data, model, loss_compute, epoch):
    start = time.time()
    total_tokens = 0.
    total_loss = 0.
    tokens = 0.
    
    for i, batch in enumerate(data):
        out = model(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
    
        if i % 50 == 1:
                elapsed = time.time() - start
                print("Epoch %d Batch: %d Loss: %f Tokens per Sec: %fs" % (epoch, i - 1, loss / batch.ntokens, (tokens.float() / elapsed / 1000.)))
                start = time.time()
                tokens = 0

    return total_loss / total_tokens

def train(data, model, criterion, optimizer):
    # set the best loss to a large number
    best_dev_loss = 1e5
    
    for epoch in range(EPOCHS):
        # train the model
        model.train()
        loss_compute = SimpleLossCompute(model.generator, criterion, optimizer)
        run_epoch(data.train_data, model, loss_compute, epoch)
        model.eval()
        
        # evaluate in dev data
        print('>>>>> Evaluate')
        dev_loss = run_epoch(data.dev_data, model, SimpleLossCompute(model.generator, criterion, None), epoch)
        print('<<<<< Evaluate loss: %f' % dev_loss)
        
        # TODO: 如果当前epoch的模型在dev集上的loss优于之前记录的最优loss则保存当前模型，并更新最优loss值
        if best_dev_loss > dev_loss:
            torch.save(model.state_dict(), SAVE_FILE)
            best_dev_loss = dev_loss
        
        
        print()
        

In [ ]:
# prepare the data

data = PrepareData(TRAIN_FILE, DEV_FILE) 
src_vocab = len(data.en_word_dict)
tgt_vocab = len(data.cn_word_dict)
print("src_vocab %d" % src_vocab)
print("tgt_vocab %d" % tgt_vocab)

model = make_model(
                    src_vocab, 
                    tgt_vocab, 
                    LAYERS, 
                    D_MODEL, 
                    D_FF,
                    H_NUM,
                    DROPOUT
                )

# training
print(">>>>>>>>>>> Start Train")
train_start = time.time()
criterion = LabelSmoothing(tgt_vocab, padding_idx = 0, smoothing= 0.0)
optimizer = NoamOpt(D_MODEL, 1, 2000, torch.optim.Adam(model.parameters(), lr=0, betas=(0.9,0.98), eps=1e-9))

train(data, model, criterion, optimizer)
print(f"<<<<<<< finished train, cost {time.time()-train_start:.4f} seconds")

In [ ]:
# import nltk
# nltk.download('punkt')

### Model Prediction

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    
    # encode
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    
    for i in range(max_len-1):
        # decode
        out = model.decode(memory, 
                           src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1)).type_as(src.data)))
        
        # log_softmax to get the probability
        prob = model.generator(out[:, -1])
        # get the predicted word and id
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys
    
def evaluate(data, model):
    
    with torch.no_grad():
        for i in range(len(data.dev_en)):
            # print the Eng sentence
            en_sent = " ".join([data.en_index_dict[word] for word in data.dev_en[i]])
            print("\n" + en_sent)
            
            # print the CN sentence
            cn_sent = " ".join([data.cn_index_dict[word] for word in data.dev_cn[i]])
            print("".join(cn_sent))
            
            # change the dev text into tensor
            src = torch.from_numpy(np.array(data.dev_en[i])).long().to(DEVICE)
            # add one dim
            src = src.unsqueeze(0)
            # attention mask
            src_mask = (src != 0).unsqueeze(-2)
            # decode and predict
            out = greedy_decode(model, src, src_mask, max_len=MAX_LENGTH, start_symbol=data.cn_word_dict["BOS"])
            
            translation = []
            
            for j in range(1, out.size(1)):
                
                sym = data.cn_index_dict[out[0, j].item()]
                
                if sym != 'EOS':
                    translation.append(sym)
                
                else:
                    break
            # print the predicted CN translation
            print("translation: %s" % " ".join(translation))
    

In [ ]:
# load the model
model.load_state_dict(torch.load(SAVE_FILE))

print(">>>>>>> start evaluate")
evaluate_start  = time.time()
evaluate(data, model)         
print(f"<<<<<<< finished evaluate, cost {time.time()-evaluate_start:.4f} seconds")